In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from sklearn.linear_model import Lasso

---

In [4]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 
              'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 
              'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 
              'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 
              'view':int}

In [5]:
sales = pd.read_csv('./data/kc_house_data.csv', dtype=dtype_dict, index_col=0)

---

Create new features by performing following transformation on inputs:

In [6]:
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(np.sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(np.sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']
sales['floors_square'] = sales['floors']*sales['floors']

- Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
- On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

---

Using the entire house dataset, learn regression weights using an L1 penalty of 5e2. Make sure to add "normalize=True" when creating the Lasso object. Refer to the following code snippet for the list of features.

In [7]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

model_all = Lasso(alpha=5e2, normalize=True).fit(sales[all_features], sales['price'])

In [8]:
np.array(all_features)[model_all.coef_ != 0]

array(['sqft_living', 'view', 'grade'], dtype='<U16')

---

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets. Download the provided csv files containing training, validation and test sets.

In [9]:
testing = pd.read_csv('./data/wk3_kc_house_test_data.csv', dtype=dtype_dict, index_col=0)
training = pd.read_csv('./data/wk3_kc_house_train_data.csv', dtype=dtype_dict, index_col=0)
validation = pd.read_csv('./data/wk3_kc_house_valid_data.csv', dtype=dtype_dict, index_col=0)

In [10]:
testing['sqft_living_sqrt'] = testing['sqft_living'].apply(np.sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(np.sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(np.sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(np.sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(np.sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(np.sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

---

Now for each l1_penalty in \[10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7\] (to get this in Python, type np.logspace(1, 7, num=13).)

- Learn a model on TRAINING data using the specified l1_penalty. Make sure to specify normalize=True in the constructor.
- Compute the RSS on VALIDATION for the current model (print or save the RSS)

In [11]:
l1_penalty_values = np.logspace(1, 7, num=13)
RSS_validation = np.zeros_like(l1_penalty_values)

for i, l1_penalty in enumerate(l1_penalty_values):
    model = Lasso(alpha=l1_penalty, normalize=True).fit(training[all_features], training.price)
    yhat = model.predict(validation[all_features])
    RSS_validation[i] = np.sum((validation.price - yhat)**2)

In [12]:
l1_penalty_v = l1_penalty_values[np.argmin(RSS_validation)]
l1_penalty_v

10.0

---

Now that you have selected an L1 penalty, compute the RSS on TEST data for the model with the best L1 penalty.

In [13]:
model_v = Lasso(alpha=l1_penalty_v, normalize=True).fit(training[all_features], training.price)

In [14]:
np.count_nonzero(model_v.coef_) + np.count_nonzero(model_v.intercept_)

15

---

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

You are going to implement a simple, two phase procedure to achieve this goal:

- Explore a large range of ‘l1_penalty’ values to find a narrow region of ‘l1_penalty’ values where models are likely to have the desired number of non-zero weights.
- Further explore the narrow region you found to find a good value for ‘l1_penalty’ that achieves the desired sparsity. Here, we will again use a validation set to choose the best value for ‘l1_penalty’.

In [15]:
max_nonzeros = 7
l1_penalty_values = np.logspace(1, 4, num=20)

- Fit a regression model with a given l1_penalty on TRAIN data. Add "alpha=l1_penalty" and "normalize=True" to the parameter list.
- Extract the weights of the model and count the number of nonzeros. Take account of the intercept as we did in #8, adding 1 whenever the intercept is nonzero. Save the number of nonzeros to a list.

In [16]:
nonzeros_values = np.zeros_like(l1_penalty_values)

for i, l1_penalty in enumerate(l1_penalty_values):
    model = Lasso(alpha=l1_penalty, normalize=True).fit(training[all_features], training.price)
    nonzeros_values[i] = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)

---

Out of this large range, we want to find the two ends of our desired narrow range of l1_penalty. At one end, we will have l1_penalty values that have too few non-zeros, and at the other end, we will have an l1_penalty that has too many non-zeros.

More formally, find:

- The largest l1_penalty that has more non-zeros than ‘max_nonzeros’ (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)Store this value in the variable ‘l1_penalty_min’ (we will use it later)
- The smallest l1_penalty that has fewer non-zeros than ‘max_nonzeros’ (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)Store this value in the variable ‘l1_penalty_max’ (we will use it later)

Hint: there are many ways to do this, e.g.:

- Programmatically within the loop above
- Creating a list with the number of non-zeros for each value of l1_penalty and inspecting it to find the appropriate boundaries.

In [17]:
idx_min = np.where(nonzeros_values > max_nonzeros)[0][-1]
idx_max = np.where(nonzeros_values < max_nonzeros)[0][0]

In [18]:
l1_penalty_min = l1_penalty_values[idx_min]
l1_penalty_max = l1_penalty_values[idx_max]

In [19]:
l1_penalty_min, l1_penalty_max

(127.42749857031335, 263.6650898730358)

In [20]:
l1_penalty_values[nonzeros_values == max_nonzeros]

array([183.29807108])

---

Exploring narrower range of l1_penalty

We now explore the region of l1_penalty we found: between ‘l1_penalty_min’ and ‘l1_penalty_max’. We look for the L1 penalty in this range that produces exactly the right number of nonzeros and also minimizes RSS on the VALIDATION set.

For l1_penalty in np.linspace(l1_penalty_min,l1_penalty_max,20):

- Fit a regression model with a given l1_penalty on TRAIN data. As before, use "alpha=l1_penalty" and "normalize=True".
- Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity equal to ‘max_nonzeros’. (Again, take account of the intercept when counting the number of nonzeros.)

In [21]:
l1_penalty_values = np.linspace(l1_penalty_min, l1_penalty_max, 20)
RSS_validation = np.zeros_like(l1_penalty_values)
nonzeros_values = np.zeros_like(l1_penalty_values)

for i, l1_penalty in enumerate(l1_penalty_values):
    model = Lasso(alpha=l1_penalty, normalize=True).fit(training[all_features], training.price)
    yhat = model.predict(validation[all_features])
    RSS_validation[i] = np.sum((validation.price - yhat)**2)
    nonzeros_values[i] = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)

In [22]:
idx_targets = np.where(nonzeros_values == max_nonzeros)[0]
idx_final = idx_targets[np.argmin(RSS_validation[idx_targets])]
l1_penalty_final = l1_penalty_values[idx_final]
l1_penalty_final

156.10909673930755

In [23]:
model_final = Lasso(alpha=l1_penalty_final, normalize=True).fit(training[all_features], training.price)

In [24]:
np.array(all_features)[model_final.coef_ != 0]

array(['bathrooms', 'sqft_living', 'waterfront', 'view', 'grade',
       'yr_built'], dtype='<U16')

---

# LASSO Solver via Coordinate Descent

Next, from Module 2 (Multiple Regression), copy and paste the ‘get_numpy_data’ function (or equivalent) that takes a data set, a list of features (e.g. \[‘sqft_living’, ‘bedrooms’\]), to be used as inputs, and a name of the output (e.g. ‘price’). This function returns a ‘features_matrix’ (2D array) consisting of first a column of ones followed by columns containing the values of the input features in the data set in the same order as the input list. It also returns an ‘output_array’ which is an array of the values of the output in the data set (e.g. ‘price’).

In [25]:
def get_numpy_data(df, features, output):
    df['constant'] = 1 # add a constant column to an DataFrame
    features = ['constant'] + features
    df_features = df.loc[:, features]
    features_matrix = df_features.values
    output_array = df.loc[:, output].values
    return features_matrix, output_array

Similarly, copy and paste the ‘predict_output’ function (or equivalent) from Module 2. This function accepts a 2D array ‘features_matrix’ and a 1D array ‘weights’ and return a 1D array ‘predictions’.

In [26]:
def predict_output(features_matrix, weights):
    predictions = features_matrix @ weights
    return predictions

In the house dataset, features vary wildly in their relative magnitude: ‘sqft_living’ is very large overall compared to ‘bedrooms’, for instance. As a result, weight for ‘sqft_living’ would be much smaller than weight for ‘bedrooms’. This is problematic because “small” weights are dropped first as l1_penalty goes up.

To give equal considerations for all features, we need to normalize features as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Write a short function called ‘normalize_features(features_matrix)’, which normalizes columns of a given feature matrix. The function should return a pair ‘(normalized_features, norms)’, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data.

In [27]:
def normalize_features(features_matrix):
    norms = np.linalg.norm(features_matrix, axis=0)
    normalized_features = features_matrix / norms
    return (normalized_features, norms)

# Review of Coordinate Descent

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include w\[0\] in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use coordinate descent: at each iteration, we will fix all weights but weight i and find the value of weight i that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than w\[i\] are held to be constant. We will optimize one w\[i\] at a time, circling through the weights multiple times.

- Pick a coordinate i
- Compute w\[i\] that minimizes the LASSO cost function
- Repeat the two steps for all coordinates, multiple times

For this assignment, we use cyclical coordinate descent with normalized features, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)    if ro[i] < -lambda/2
w[i] = ├ 0                     if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)    if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```
Note that we do not regularize the weight of the constant feature (intercept) w[0], so, for this weight, the update is simply:
```
w[0] = ro[i]
```

# Effect of L1 penalty

Consider a simple model with 2 features: ‘sqft_living’ and ‘bedrooms’. The output is ‘price’.

- First, run get_numpy_data() (or equivalent) to obtain a feature matrix with 3 columns (constant column added). Use the entire ‘sales’ dataset for now.
- Normalize columns of the feature matrix. Save the norms of original features as ‘norms’.
- Set initial weights to \[1,4,1\].
- Make predictions with feature matrix and initial weights.
- Compute values of ro\[i\], where

In [28]:
features = ['sqft_living', 'bedrooms']
features_matrix, output = get_numpy_data(sales, features, 'price')
normalized_features, norms = normalize_features(features_matrix)

In [29]:
initial_weights = np.array([1, 4, 1])
prediction = predict_output(normalized_features, initial_weights)

In [30]:
p = normalized_features.shape[1]
ρ = np.zeros(p)

for i in range(p):
    feature_i = normalized_features[:,i]
    ρ[i] = np.sum(feature_i * (output - prediction + initial_weights[i]*feature_i))

In [31]:
print('{:.4e}'.format(ρ[1] * 2))

1.7588e+08


In [32]:
print('{:.4e}'.format(ρ[2] * 2))

1.6193e+08


So we can say that ro\[i\] quantifies the significance of the i-th feature: the larger ro\[i\] is, the more likely it is for the i-th feature to be retained.

# Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [33]:
def lasso_coordinate_descent_step(i, features_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(features_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    feature_i = features_matrix[:,i]
    ro_i = np.sum(feature_i * (output - prediction + weights[i]*feature_i))
    
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i
    # reduce magnitude of coefficients
    elif ro_i < -l1_penalty/2:
        new_weight_i = ro_i + l1_penalty/2
    elif ro_i > l1_penalty/2:
        new_weight_i = ro_i - l1_penalty/2
    else:
        new_weight_i = 0.
    
    return new_weight_i

If you are using Numpy, test your function with the following snippet:

In [34]:
# should print 0.425558846691
print(lasso_coordinate_descent_step(1, 
                                    np.array([[3./np.sqrt(13),1./np.sqrt(10)],
                                              [2./np.sqrt(13),3./np.sqrt(10)]]), 
                                    np.array([1., 1.]), np.array([1., 4.]), 0.1))

0.4255588466910251


# Cyclical coordinate descent

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:

- As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
- After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to the previous step.

Return weights

The function should accept the following parameters:

- Feature matrix
- Output array
- Initial weights
- L1 penalty
- Tolerance

In [35]:
def lasso_cyclical_coordinate_descent(features_matrix, output, initial_weights, l1_penalty, tolerance):
    p = features_matrix.shape[1]
    Δcoordinates = np.ones(p) * (tolerance + 1)
    weights = initial_weights
    
    while np.max(np.abs(Δcoordinates) >= tolerance):
        for i in range(p):
            new_weight_i = lasso_coordinate_descent_step(i, features_matrix, output, weights, l1_penalty)
            Δcoordinates[i] = new_weight_i - weights[i]
            weights[i] = new_weight_i
    
    return weights

Let us now go back to the simple model with 2 features: ‘sqft_living’ and ‘bedrooms’. Using ‘get_numpy_data’ (or equivalent), extract the feature matrix and the output array from from the house dataframe. Then normalize the feature matrix using ‘normalized_features()’ function.

Using the following parameters, learn the weights on the sales dataset.

- Initial weights = all zeros
- L1 penalty = 1e7
- Tolerance = 1.0

In [36]:
initial_weights = np.zeros(normalized_features.shape[1])
l1_penalty = 1e7
tolerance = 1.0

weights = lasso_cyclical_coordinate_descent(normalized_features, output, 
                                            initial_weights, l1_penalty, tolerance)
print(weights)

[21624997.95951909 63157247.20788956        0.        ]


In [37]:
np.array(features)[(weights == 0)[1:]]

array(['bedrooms'], dtype='<U11')

In [38]:
prediction = predict_output(normalized_features, weights)
RSS = np.sum((sales.price - prediction)**2)
print('{:.4e}'.format(RSS))

1.6305e+15


# Evaluating LASSO fit with more features

Create a normalized feature matrix from the TRAINING data with the following set of features.

- bedrooms, bathrooms, sqft_living, sqft_lot, floors, waterfront, view, condition, grade, sqft_above, sqft_basement, yr_built, yr_renovated

Make sure you store the norms for the normalization, since we’ll use them later.

In [39]:
df_Te = pd.read_csv('./data/kc_house_test_data.csv', dtype=dtype_dict, index_col=0)
df_Tr = pd.read_csv('./data/kc_house_train_data.csv', dtype=dtype_dict, index_col=0)

In [40]:
features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 
            'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']
features_matrix, output = get_numpy_data(df_Tr, features, 'price')
normalized_features, norms = normalize_features(features_matrix)

First, learn the weights with l1_penalty=1e7, on the training data. Initialize weights to all zeros, and set the tolerance=1. Call resulting weights’ weights1e7’, you will need them later.

In [41]:
initial_weights = np.zeros(normalized_features.shape[1])
l1_penalty = 1e7
tolerance = 1.0

weights1e7 = lasso_cyclical_coordinate_descent(normalized_features, output, 
                                               initial_weights, l1_penalty, tolerance)
print(weights1e7)

[24429600.23440312        0.                0.         48389174.77154896
        0.                0.          3317511.21492165  7329961.81171425
        0.                0.                0.                0.
        0.                0.        ]


In [42]:
np.array(features)[(weights1e7 != 0)[1:]]

array(['sqft_living', 'waterfront', 'view'], dtype='<U13')

Next, learn the weights with l1_penalty=1e8, on the training data. Initialize weights to all zeros, and set the tolerance=1. Call resulting weights ‘weights1e8’, you will need them later.

In [43]:
initial_weights = np.zeros(normalized_features.shape[1])
l1_penalty = 1e8
tolerance = 1.0

weights1e8 = lasso_cyclical_coordinate_descent(normalized_features, output, 
                                               initial_weights, l1_penalty, tolerance)
print(weights1e8)

[71114625.71488702        0.                0.                0.
        0.                0.                0.                0.
        0.                0.                0.                0.
        0.                0.        ]


Finally, learn the weights with l1_penalty=1e4, on the training data. Initialize weights to all zeros, and set the tolerance=5e5. Call resulting weights ‘weights1e4’, you will need them later. (This case will take quite a bit longer to converge than the others above.)

In [44]:
initial_weights = np.zeros(normalized_features.shape[1])
l1_penalty = 1e4
tolerance = 5e5

weights1e4 = lasso_cyclical_coordinate_descent(normalized_features, output, 
                                               initial_weights, l1_penalty, tolerance)
print(weights1e4)

[ 78564738.34156762 -22097398.92430532  12791071.87278517
  93808088.09281193  -2013172.75704954  -4219184.93265014
   6482842.81753506   7127408.53480689   5001664.8546964
  14327518.43714051 -15770959.15237397  -5159591.22213147
 -84495341.7684364    2824439.49703683]


In [45]:
np.array(features)[(weights1e4 != 0)[1:]]

array(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated'], dtype='<U13')

# Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights. To use these weights on a test set, we must normalize the test data in the same way. Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data:

In this case, we must scale the resulting weights so that we can make predictions with original features:

- Store the norms of the original features to a vector called ‘norms’.
- Run Lasso on the normalized features and obtain a ‘weights’ vector.
- Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply weights_normalized to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (‘weights1e4’, ‘weights1e7’, ‘weights1e8’).

In [46]:
normalized_weights1e4 = weights1e4 / norms
normalized_weights1e7 = weights1e7 / norms
normalized_weights1e8 = weights1e8 / norms

In [47]:
normalized_weights1e7[3]

161.31745764611762

Let's now evaluate the three models on the test data. Extract the feature matrix and output array from the TEST set. But this time, do NOT normalize the feature matrix. Instead, use the normalized version of weights to make predictions.

Compute the RSS of each of the three normalized weights on the (unnormalized) feature matrix.

In [48]:
features_matrix_test, output_test = get_numpy_data(df_Te, features, 'price')

In [49]:
RSS1e4 = np.sum((output_test - predict_output(features_matrix_test, normalized_weights1e4))**2)
RSS1e7 = np.sum((output_test - predict_output(features_matrix_test, normalized_weights1e7))**2)
RSS1e8 = np.sum((output_test - predict_output(features_matrix_test, normalized_weights1e8))**2)

In [50]:
print('{:.4e}'.format(RSS1e4))
print('{:.4e}'.format(RSS1e7))
print('{:.4e}'.format(RSS1e8))

2.2846e+14
2.7596e+14
5.3717e+14
